# 05b – Data Cleansing: Population by Age

**Author:** Roberto Chiaiese  
**Project:** CovidReporting Data Warehouse  

##  Overview
This notebook processes and reshapes the population dataset to build the **dimCountry** and **dimPopulationByAge** dimension tables.  
It consolidates demographic information for each country.

##  Steps Performed
1. **Load data** from `staging.population_by_age`.  
2. **Unpivot yearly columns** (2008–2019) and compute the most recent population value.  
3. **Merge with country codes** (2-digit).  
4. **Calculate age group distributions** (0–14, 15–24, 25–49, 50–64, 65–79, 80+).  
5. **Create standardized columns** matching the dimension schema.  
6. **Export results** to populate:
   - `dimCountry`  
  
## Output Tables
**dimCountry**
- `country_key`, `country_name`, `country_code`, `age_0_to_14`, `age_15_to_24`, `age_25_to_49`, `age_50_to_64`, `age_65_to_79`, `age_80_to_MAX`



In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('/home/jovyan/datawarehouse/staging_layer/dataset/population_by_age.tsv', delimiter = '\t')

In [9]:
#the table has clearly a problem about the first column and all the other column names have a space at the end, it's necessary to remove it by using rstripe() function
df.columns

Index(['indic_de,geo\time', '2008 ', '2009 ', '2010 ', '2011 ', '2012 ',
       '2013 ', '2014 ', '2015 ', '2016 ', '2017 ', '2018 ', '2019 '],
      dtype='object')

In [10]:
# extract the data from the first column to obtain the 'indic_de' and 'geo' columns
list = df['indic_de,geo\\time']
list_series = list.str.split(',')
list_series =  list_series.tolist()
indic_de = []
geo = []

for lst in list_series:
    indic_de.append(lst[0])
    geo.append(lst[1][0:2])

In [15]:
#insert the lists into the dataframe
df[['indic_de','geo']]= pd.DataFrame({'indic_de':indic_de, 'geo':geo})
df.head()

,"indic_de,geo\time",2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,indic_de,geo
0,"PC_Y0_14,AD",14.6,14.5,14.5,15.5,15.5,15.5,:,:,:,:,:,13.9,PC_Y0_14,AD
1,"PC_Y0_14,AL",24.1,23.3,22.5,21.6,20.7,20.1,19.6,19.0,18.5,18.2,17.7,17.2,PC_Y0_14,AL
2,"PC_Y0_14,AM",19.0,18.6,18.3,:,:,:,:,19.4,19.6,20.0,20.2,20.2,PC_Y0_14,AM
3,"PC_Y0_14,AT",15.4,15.1,14.9,14.7,14.6,14.4,14.3,14.3,14.3,14.4,14.4,14.4,PC_Y0_14,AT
4,"PC_Y0_14,AZ",23.2,22.6,22.6,22.3,22.2,22.3,22.4,22.4,22.5,22.6,22.6,22.4,PC_Y0_14,AZ


In [ ]:
# drop the wrong column
df = df.drop('indic_de,geo\\time', axis =1)
df.head()

In [18]:
#clean the columns name removing excessive spaces
column_names=[]
for i in df.columns.tolist():
    column_names.append(i.rstrip())

df.columns = [i.rstrip() for i in df.columns]
df.head()

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,indic_de,geo
0,14.6,14.5,14.5,15.5,15.5,15.5,:,:,:,:,:,13.9,PC_Y0_14,AD
1,24.1,23.3,22.5,21.6,20.7,20.1,19.6,19.0,18.5,18.2,17.7,17.2,PC_Y0_14,AL
2,19.0,18.6,18.3,:,:,:,:,19.4,19.6,20.0,20.2,20.2,PC_Y0_14,AM
3,15.4,15.1,14.9,14.7,14.6,14.4,14.3,14.3,14.3,14.4,14.4,14.4,PC_Y0_14,AT
4,23.2,22.6,22.6,22.3,22.2,22.3,22.4,22.4,22.5,22.6,22.6,22.4,PC_Y0_14,AZ


In [19]:
# reorder the columns
df = df[['indic_de', 'geo','2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]

In [23]:
# convert all the records of the date columns to numbers ( float)
df.replace(to_replace=': ', value=np.nan, inplace = True)

cols = ['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

for col in cols:
    df[col] = df[col].replace(r'[^0-9\.-]', '', regex=True)

for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.to_csv('population_raw.csv', index=False)

In [122]:
country_lookup = pd.read_csv('/home/jovyan/datawarehouse/lookup/country_lookup.csv')


In [123]:
df_merged = pd.merge(df, country_lookup, left_on='geo', right_on='country_code_2_digit')

In [133]:
df_merged.head()

,country,country_code_2_digit,country_code_3_digit,population,indic_de,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Andorra,AD,AND,76177,PC_Y0_14,14.6,14.5,14.5,15.5,15.5,15.5,NaN,NaN,NaN,NaN,NaN,13.9
1,Andorra,AD,AND,76177,PC_Y15_24,24.1,23.3,22.5,21.6,20.7,20.1,19.6,19.0,18.5,18.2,17.7,17.2
2,Andorra,AD,AND,76177,PC_Y25_49,19.0,18.6,18.3,NaN,NaN,NaN,NaN,19.4,19.6,20.0,20.2,20.2
3,Andorra,AD,AND,76177,PC_Y50_64,15.4,15.1,14.9,14.7,14.6,14.4,14.3,14.3,14.3,14.4,14.4,14.4
4,Andorra,AD,AND,76177,PC_Y65_79,23.2,22.6,22.6,22.3,22.2,22.3,22.4,22.4,22.5,22.6,22.6,22.4


In [129]:
df_merged = df_merged[['country', 'country_code_2_digit', 'country_code_3_digit', 'population', 'indic_de', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]

In [134]:
df_merged.head()

,country,country_code_2_digit,country_code_3_digit,population,indic_de,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Andorra,AD,AND,76177,PC_Y0_14,14.6,14.5,14.5,15.5,15.5,15.5,NaN,NaN,NaN,NaN,NaN,13.9
1,Andorra,AD,AND,76177,PC_Y15_24,24.1,23.3,22.5,21.6,20.7,20.1,19.6,19.0,18.5,18.2,17.7,17.2
2,Andorra,AD,AND,76177,PC_Y25_49,19.0,18.6,18.3,NaN,NaN,NaN,NaN,19.4,19.6,20.0,20.2,20.2
3,Andorra,AD,AND,76177,PC_Y50_64,15.4,15.1,14.9,14.7,14.6,14.4,14.3,14.3,14.3,14.4,14.4,14.4
4,Andorra,AD,AND,76177,PC_Y65_79,23.2,22.6,22.6,22.3,22.2,22.3,22.4,22.4,22.5,22.6,22.6,22.4


In [ ]:
#compute the mean of the rows from 2008 to 2019 columns
df_merged['mean_value'] = df_merged[cols].mean(axis=1)


In [140]:
#pivot (partially) the table
df_pivoted = df_grouped.pivot_table(aggfunc='first', index=['country', 'country_code_2_digit', 'country_code_3_digit', 'population'], columns='indic_de', values='mean_value').reset_index()

In [141]:
df_pivoted = df_pivoted.rename(columns={'country_code_2_digit':'country_id', 'country_code_3_digit':'country_code'})

indic_de,country,country_code_2_digit,country_code_3_digit,population,age_0_to_14,age_15_to_24,age_25_to_49,age_50_to_64,age_65_to_79,age_80_to_MAX
0,Albania,AL,ALB,2862427,13.7,15.7,15.0,16.7,15.0,13.4
1,Andorra,AD,AND,76177,14.9,20.2,19.4,14.6,22.5,17.0
2,Armenia,AM,ARM,2963234,17.4,15.2,15.3,15.7,14.5,15.0
3,Austria,AT,AUT,8858775,15.6,15.3,15.6,16.4,18.5,18.4
4,Azerbaijan,AZ,AZE,10139175,18.1,14.9,14.6,21.1,20.3,13.8


In [ ]:
#select the desired fields
df_pivoted = df_pivoted[['country_id','country_code':'','country','population','age_0_to_14','age_15_to_24','age_25_to_49','age_50_to_64','age_65_to_79','age_80_to_MAX']]

In [35]:
df_pivoted.to_csv('/home/jovyan/datawarehouse/core_layer/processed/dimCountry.csv', index=False)